Dieser Quellcode ist Bestandteil der Dissertation von Ines Reinecke
vorgelegt am 11.07.2023 an der Technischen Universität Dresden, Medizinische Fakultät

Dieses Script enthält:
* Einlesen der Daten DS-Med als Eingangsgröße, nach Durchführung der Maßnahmen zur Verbesserung der Datenstruktur, im OMOP Format
* Einlesen der ATC Codes Version 2022 - mit den entsprechenden ATC Beschreibungen in Deutsch
* Zusammenführen der Medikationsverordnungen mit den ATC Codes und den Beschreibungen
* Generieren eines Streudiagramms mit der Bibliothek Bokeh, interaktiv

In [15]:
import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz, process
from typing import List
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3, venn3_circles
from pandas_profiling import ProfileReport
import warnings
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

import networkx as nx
from bokeh.models import *
from bokeh.io import output_file, show
from bokeh.plotting import *
from bokeh.layouts import layout, column
from bokeh.events import ButtonClick
from platform import python_version
from bokeh.models import LogTicker, FuncTickFormatter

warnings.filterwarnings('ignore')

In [16]:
# Dateien einlesen

# Einlesen DS-Med, verbesserte Datenstruktur

drug_exposures = pd.read_csv('../data_results/02_data_to_omop+terminology_results/Schritt-2-DQD-DS-Med.csv', skipinitialspace=True, low_memory=False, lineterminator='\n').fillna(str())

# Einlesen ATC Codes aus OMOP Vokabulare
atc_all_omop = pd.read_csv('../data_in/atc_all_omop.csv', sep=',')
atc_all_omop = atc_all_omop[["concept_id","concept_name","concept_code"]]

# Einlesen aller ATC nach RxNorm Mappings der Beziehungstypen ATC - RxNorm pr lat, ATC - RxNorm pr up, ATC - RxNorm sec lat, ATC - RxNorm sec up   
#atc_rxnorm_relationship = pd.read_csv('/Users/reineckin/work/dev/python_scripts/Diss_OMOP_Data/data_in/ATC-RxNorm-Rel.csv', sep='\t')
atc_rxnorm_relationship = pd.read_csv('../data_in/ATC-RxNorm-Rel.csv', sep='\t')

In [17]:
# Gruppierung aller concept_ids für ATC codes mit den Beziehungstypen der Mappings
atc_rxnorm_relationship_grouped = atc_rxnorm_relationship.groupby(['concept_id_atc','code_atc', 'name_atc', 'rel_id']).size().reset_index(name='count')
    
# Umformung des DataFrames, wobei Spalten für jeden der 4 Beziehungstypen erstellt werden und ein Wert hinzugefügt wird, der die Anzahl der Beziehungen für jeden ATC-Code angibt. Das Ergebnis ist nur eine Zeile pro ATC-Code.
atc_rxnorm_relationship_grouped_unmelted=atc_rxnorm_relationship_grouped.pivot(index=['concept_id_atc','code_atc', 'name_atc'], columns='rel_id')
    
# Reset Index, Generieren eines Dataframes nach Umformung
atc_rxnorm_relationship_grouped_unmelted = atc_rxnorm_relationship_grouped_unmelted['count'].reset_index()
atc_rxnorm_relationship_grouped_unmelted.columns.name = None

#Änderung von Datentypen für Spalten
atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr lat'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr lat'].astype('Int64')
atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr up'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr up'].astype('Int64')
atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec lat'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec lat'].astype('Int64')
atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec up'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec up'].astype('Int64')
atc_rxnorm_relationship_grouped_unmelted['Maps to'] = atc_rxnorm_relationship_grouped_unmelted['Maps to'].astype('Int64')
    
prepared_atc_rxnorm_relationship = atc_rxnorm_relationship_grouped_unmelted
    


In [18]:
#group the relationships based on the combinations in numbers for the different relationship types

prepared_atc_rxnorm_relationship_grouped = prepared_atc_rxnorm_relationship.groupby(["ATC - RxNorm pr lat","ATC - RxNorm pr up","ATC - RxNorm sec lat","ATC - RxNorm sec up","Maps to"],dropna=False).size().to_frame('size')



In [19]:
prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to'].isnull()) & (prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat'] == 1)].to_csv('mapsTo_0_ATCRxNormprLat_1.csv')

prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat'].isnull()) & (prepared_atc_rxnorm_relationship['Maps to'] == 1)].to_csv('mapsTo_1_ATCRxNormprLat_0.csv')

print("Maps to is exactly 1: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']==1)]))
print("Maps to is greater than 1: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>1)]))
print("Maps to is greater than 2: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>2)]))
print("Maps to is greater than 3: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>3)]))
print("Maps to is greater than 4: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>4)]))
print("Maps to is greater than 5: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>5)]))
print("Maps to is greater than 6: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>6)]))
print("Maps to is greater than 7: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>7)]))
print("Maps to is greater than 8: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>8)]))
print("Maps to is greater than 9: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to']>9)]))


Maps to is exactly 1:  4285
Maps to is greater than 1:  393
Maps to is greater than 2:  103
Maps to is greater than 3:  26
Maps to is greater than 4:  13
Maps to is greater than 5:  10
Maps to is greater than 6:  8
Maps to is greater than 7:  6
Maps to is greater than 8:  5
Maps to is greater than 9:  4


In [20]:

#Zusammenführen von den Medikationsverordnungen mit den Mappings von ATC nach RxNorm

drug_exposures_merged_rxnorm_relationships = pd.merge(drug_exposures, prepared_atc_rxnorm_relationship, left_on='drug_source_concept_id', right_on='concept_id_atc', how="left", indicator=True)

#Limitierung der Medikationsverordnungen auf die mit einer validen concept_id ungleich 0
drug_exposures_merged_rxnorm_relationships_valid_concepts_ATC_only = drug_exposures_merged_rxnorm_relationships.loc[drug_exposures_merged_rxnorm_relationships['drug_source_concept_id']!=0]

drug_exposures_with_any_rxnorm_only = drug_exposures_merged_rxnorm_relationships_valid_concepts_ATC_only.copy()

In [21]:

#Gruppierung der Medikamentenverordnungen/ATC Codes und Beziehungen und Zählen der Häufigkeit für ATC-Codes in einer neuen Spalte "count".
drug_expsoures_with_rxnorm_grouped = drug_exposures_with_any_rxnorm_only.groupby(['drug_source_concept_id','code_atc','name_atc', 'ATC - RxNorm pr lat','ATC - RxNorm pr up','ATC - RxNorm sec lat','ATC - RxNorm sec up','Maps to'],dropna=False).size().reset_index(name='count').sort_values('count', ascending=False)

# Alle Nan Werte mit 0 ersetzen
drug_expsoures_with_rxnorm_grouped = drug_expsoures_with_rxnorm_grouped.fillna(0)

# Reset des Index
drug_expsoures_with_rxnorm_grouped = drug_expsoures_with_rxnorm_grouped.reset_index()

# Hinzufügen der Spalte "color" mit den entsprechenden Farbwerten
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["Maps to"] >= 1, 'color'] = "#2c35b3"
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["Maps to"] == 0, 'color'] = "#a3001e"



In [22]:
# Neue Spalte hinzufügen, mit der Summe der Anzahl der Mappings pro Beziehungstyp
drug_expsoures_with_rxnorm_grouped['sum_map'] = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr lat']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr up']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec lat']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec up']

In [23]:
 # Zusammenführen der ATC Codes mit den Namen und den Beziehungstypen des Mappings nach RxNorm
drug_expsoures_with_rxnorm_grouped_atc_full = pd.merge(drug_expsoures_with_rxnorm_grouped, atc_all_omop, left_on='drug_source_concept_id', right_on='concept_id', how="left")
drug_expsoures_with_rxnorm_grouped_atc_full.loc[drug_expsoures_with_rxnorm_grouped_atc_full['code_atc'] == 0, 'code_atc'] = drug_expsoures_with_rxnorm_grouped_atc_full.loc[drug_expsoures_with_rxnorm_grouped_atc_full['code_atc'] == 0, 'concept_code']
drug_expsoures_with_rxnorm_grouped_atc_full.loc[drug_expsoures_with_rxnorm_grouped_atc_full['name_atc'] == 0, 'name_atc'] = drug_expsoures_with_rxnorm_grouped_atc_full.loc[drug_expsoures_with_rxnorm_grouped_atc_full['name_atc'] == 0, 'concept_name']

In [24]:

# Reduktion des Dataframes auf 3 Spalten
drug_expsoures_with_rxnorm_grouped_atc_full = drug_expsoures_with_rxnorm_grouped_atc_full.drop(['concept_id', 'concept_name', 'concept_code'], axis=1)

In [25]:
#Helfer Dataframe für das Streudiagramm generieren

x = drug_expsoures_with_rxnorm_grouped_atc_full['count']
y = drug_expsoures_with_rxnorm_grouped_atc_full['sum_map']
y_help = drug_expsoures_with_rxnorm_grouped_atc_full['sum_map']
ATC_code = drug_expsoures_with_rxnorm_grouped_atc_full['code_atc']
ATC_name = drug_expsoures_with_rxnorm_grouped_atc_full['name_atc']
color = drug_expsoures_with_rxnorm_grouped_atc_full['color']
ATC_RxNorm_pr_lat = drug_expsoures_with_rxnorm_grouped_atc_full['ATC - RxNorm pr lat']
ATC_RxNorm_pr_up = drug_expsoures_with_rxnorm_grouped_atc_full['ATC - RxNorm pr up']
ATC_RxNorm_sec_lat = drug_expsoures_with_rxnorm_grouped_atc_full['ATC - RxNorm sec lat']
ATC_RxNorm_sec_up = drug_expsoures_with_rxnorm_grouped_atc_full['ATC - RxNorm sec up']
Maps_to = drug_expsoures_with_rxnorm_grouped_atc_full['Maps to']


df = pd.DataFrame({'ATC_code': ATC_code, 'ATC_name': ATC_name, 'frequency': x, 'TOTAL_RxNorm_mappings': y,'y_helper': y_help, 'ATC_RxNorm_pr_lat': ATC_RxNorm_pr_lat, 'ATC_RxNorm_pr_up': ATC_RxNorm_pr_up, 'ATC_RxNorm_sec_lat': ATC_RxNorm_sec_lat, 'ATC_RxNorm_sec_up': ATC_RxNorm_sec_up, 'Maps_to': Maps_to, 'color': color})


In [26]:
#das hier sind nur die Medikationsverordnungen ohne explizites Mapping nach RxNorm über Maps to, die haben trotzdem andere Beziehungen nach RxNorm.. deshalb ist das Ergebnis geringer als erwartet
#die anderen ATC Codes werden nach dem Merge mit den Beziehungen schon gelöscht, weil bei dem Merge danach gecheckt wird und nur die behalten werden die beim Indicator "both" stehen haben
print("Anzahl Medikationsverordnungen OHNE explizites Mapping nach RxNorm: ",df["frequency"].loc[df["Maps_to"]==0].sum())
print("Anzahl Medikationsverordnungen mit expliziten Mapping nach RxNorm: ",df["frequency"].loc[df["Maps_to"]==1].sum())
print("Anzahl Medikationsverordnungen mit ZWEI expliziten Mappings nach RxNorm: ",df["frequency"].loc[df["Maps_to"]==2].sum())
print("Anzahl Medikationsverordnungen mit DREI expliziten Mappings nach RxNorm: ",df["frequency"].loc[df["Maps_to"]==3].sum())
print("Anzahl Medikationsverordnungen mit mehr als DREI expliziten Mappings nach RxNorm: ",df["frequency"].loc[df["Maps_to"]>3].sum())

Anzahl Medikationsverordnungen OHNE explizites Mapping nach RxNorm:  336729
Anzahl Medikationsverordnungen mit expliziten Mapping nach RxNorm:  1155692
Anzahl Medikationsverordnungen mit ZWEI expliziten Mappings nach RxNorm:  13632
Anzahl Medikationsverordnungen mit DREI expliziten Mappings nach RxNorm:  6
Anzahl Medikationsverordnungen mit mehr als DREI expliziten Mappings nach RxNorm:  0


In [27]:

# Änderung Datentpy von Spalten des Helfer Dataframes
df[['TOTAL_RxNorm_mappings','y_helper']] = df[['TOTAL_RxNorm_mappings','y_helper']].astype(float)

# Änderung aller y Werte von 0 auf 0,7 da ansonsten die betreffenden ATC Codes ohne Mapping nach RxNorm nicht in dem Streudiagramm zu finden sind
df.loc[df['y_helper'] == 0, 'y_helper'] = 0.7


In [28]:
# Generierung des Streudiagrams

plot_source = ColumnDataSource(df)

sourcetext = ColumnDataSource(data={'ATC_code': [], 'ATC_name': [], 'frequency': [], 'TOTAL_RxNorm_mappings': [],'y_helper': [], 'ATC_RxNorm_pr_lat': [], 'ATC_RxNorm_pr_up': [], 'ATC_RxNorm_sec_lat': [], 'ATC_RxNorm_sec_up': [],'Maps_to': [], 'color': []})
    
#Callback Funktion, um die gefilterten ATC Codes als Punkte dann im Streudiagramm hervorzuheben
clearbutton = Button(label="Clear highlighted codes", button_type="primary", width=50)
clearbuttoncallback = CustomJS(args=dict(sourcetext=sourcetext), code="""
    var d2 = sourcetext.data;
            d2['ATC_code']=[]
            d2['ATC_name']=[]
            d2['frequency']=[]
            d2['TOTAL_RxNorm_mappings']=[]
            d2['y_helper']=[]
            d2['ATC_RxNorm_pr_lat']=[]
            d2['ATC_RxNorm_pr_up']=[]
            d2['ATC_RxNorm_sec_lat']=[]
            d2['ATC_RxNorm_sec_up']=[]
            d2['Maps_to']=[]
            d2['color']=[]
    sourcetext.data = d2;        
    sourcetext.change.emit();""")
clearbutton.js_on_event(ButtonClick, clearbuttoncallback)

# Generierung des Textfelds für die Suche
search = TextInput(value = "", placeholder = "A1...", width=200)


# Callback für die Handhabung des Suchstrings und die Anwendung des Filters
callback_search = CustomJS(args=dict(source=plot_source, sourcetext=sourcetext), code="""
    var label = cb_obj.value.toLowerCase();
    var d1 = source.data;
    var d2 = sourcetext.data;
    for (var i = 0; i < d1['ATC_code'].length; i++) {
        if (d1['ATC_code'][i].toLowerCase().includes(label)|d1['ATC_name'][i].toLowerCase().includes(label)){
            d2['ATC_code'].push(d1['ATC_code'][i]);
            d2['ATC_name'].push(d1['ATC_name'][i]);
            d2['frequency'].push(d1['frequency'][i]);
            d2['TOTAL_RxNorm_mappings'].push(d1['TOTAL_RxNorm_mappings'][i]);
            d2['y_helper'].push(d1['y_helper'][i]);
            d2['ATC_RxNorm_pr_lat'].push(d1['ATC_RxNorm_pr_lat'][i]);
            d2['ATC_RxNorm_pr_up'].push(d1['ATC_RxNorm_pr_up'][i]);
            d2['ATC_RxNorm_sec_lat'].push(d1['ATC_RxNorm_sec_lat'][i]);
            d2['ATC_RxNorm_sec_up'].push(d1['ATC_RxNorm_sec_up'][i]);
            d2['Maps_to'].push(d1['Maps_to'][i]);
            d2['color'].push(d1['color'][i]);
        }
    }
    sourcetext.data = d2;
    sourcetext.change.emit();
""")

# Callback für Textfeld zur Suche hinzufügen
search.js_on_change('value', callback_search) 


fig = figure(tools="pan,wheel_zoom,box_zoom,reset,save, tap, lasso_select",
         x_axis_type="log",y_axis_type="log", plot_width=1000, plot_height=900, x_axis_label='Summe der Medikationsverordnungen (logarithmische Skala)', y_axis_label='Anzahl ATC nach RxNorm mappings')


fig.add_layout(Legend())

r1 = fig.circle(x='frequency', y='y_helper', color='color', size=6,alpha=0.5, source=plot_source, legend_label="ATC Code (blau = mind. ein Maps to nach RxNorm, rot = kein Maps to nach RxNorm)")
r2 = fig.circle(x='frequency', y='y_helper',  color='color', size=12, alpha=0.8, source=sourcetext)


fig.legend.location = "top_left"
fig.xaxis.axis_label_text_font_size = "12pt"
fig.yaxis.axis_label_text_font_size = "12pt"
fig.title.text_font_size = '14pt'
fig.yaxis.major_label_text_font_size = "12pt"
fig.xaxis.major_label_text_font_size = "12pt"

hover = HoverTool(tooltips=[("ATC code", "@ATC_code"),("ATC name", "@ATC_name"),("frequency", "@frequency"), ("TOTAL RxNorm mappings", "@TOTAL_RxNorm_mappings"), ("RxNorm pr lat mappings", "@ATC_RxNorm_pr_lat"), ("RxNorm pr up mappings", "@ATC_RxNorm_pr_up"), ("RxNorm sec lat mappings", "@ATC_RxNorm_sec_lat"), ("RxNorm sec up mappings", "@ATC_RxNorm_sec_up"), ("Maps to", "@Maps_to")])
hover.renderers =[r1]


fig.add_tools(hover)


columns = [
    TableColumn(field="ATC_code", title="ATC Code",
                formatter=StringFormatter(font_style="bold")),
    TableColumn(field="ATC_name", title="Wirkstoff"),
    TableColumn(field="frequency", title="Anzahl gesamt",
                editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0")),
    TableColumn(field="TOTAL_RxNorm_mappings", title="Gesamtzahl Mappings nach RxNorm", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0")),
    TableColumn(field="Maps_to", title="Maps to", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0"))
    ]

data_table = DataTable(source=sourcetext, columns=columns, editable=False, sortable=True, selectable="checkbox", width=800, height=100)
#div_header = Div(text="<br>Distribution of medication orders as ATC code by frequency and number of total ATC to RxNorm ingredient mappings<br><br>", style={'font-size': '150%', 'color': 'steelblue'}, width=1200)
div_table = Div(text="<br>   <br>", style={'font-size': '100%', 'color': 'black'})
div_search = Div(text="<br>Eingabe ATC Code oder Wirkstoff:", style={'font-size': '100%', 'color': 'black'})
div_blank = Div(text="")

search_col = column(div_search, search)
find_tools = column(search_col, clearbutton,)
table_col = column(div_table, data_table)

    
output_file('../data_results/03_data_transparency_results/streu_atc_rxnorm.html')

l=layout([
    [find_tools, table_col],
    [fig],
    [Div(text="<br><br><br>")]])
show(l)

